In [52]:
import pandas as pd
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import LocalOutlierFactor
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, classification_report, roc_auc_score, confusion_matrix, roc_curve, auc
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

In [70]:
zv_df = pd.read_csv('data/zwangsversteigerungen.csv')
wohnungen_df = zv_df[zv_df['typ'] == 'Wohnung']

immo_df = pd.read_csv('data/immo_data.csv')
immo_df = immo_df[
    # I want to focus only on apartments, an apartment has specific characteristics which I will define on my own here
    # also I want to focus on apartments where it would be realistic to buy them
    (immo_df['yearConstructed'] >= wohnungen_df['baujahr'].min()) &
    (immo_df['baseRent'] <= np.percentile(immo_df['baseRent'], 99.9)) & # filter out > 4800
    (immo_df['livingSpace'] <= np.percentile(immo_df['livingSpace'], 99.9)) & # filter out > 280m^2
    (immo_df['noRooms'] <= 10)
]
immo_df = immo_df[['regio1', 'balcony', 'baseRent', 'yearConstructed', 'noRooms', 'livingSpace', 'garden']]
immo_df = immo_df.dropna()
immo_df['regio1'] = immo_df['regio1'].str.replace('_', '-')

In [71]:
immo_df = immo_df.sample(15000)

In [73]:
X = immo_df[['regio1', 'balcony', 'baseRent', 'yearConstructed', 'noRooms', 'livingSpace', 'garden']]
y = immo_df['baseRent']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

preprocessor = ColumnTransformer(
    transformers=[
        ('regio1', OneHotEncoder(), ['regio1']),
        ('bool_cols', 'passthrough', ['balcony', 'garden']),
        ('num_cols', 'passthrough', ['yearConstructed', 'noRooms', 'livingSpace']),
    ]
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

param_grid = [
    #{
    #    'regressor': [LinearRegression()]
    #},
    #{
    #    'regressor': [SVR()],
    #    'regressor__kernel': ['rbf'],
    #    'regressor__C': [1, 10, 30],
    #    'regressor__epsilon': [0.1, 0.2]
    #},
    #{
    #    'regressor': [DecisionTreeRegressor(random_state=2024)],
    #    'regressor__max_depth': [5, 10, 15, 50],
    #    'regressor__min_samples_split': [2, 5, 10, 50]
    #},
    #{
    #    'regressor': [RandomForestRegressor(random_state=2024)],
    #    'regressor__n_estimators': [50, 100, 300],
    #    'regressor__max_depth': [10, 20]
    #},
    #{
    #    'regressor': [GradientBoostingRegressor(random_state=2024)],
    #    'regressor__n_estimators': [100, 200],
    #    'regressor__learning_rate': [0.1, 0.15]
    #},
    #{
    #    'regressor': [MLPRegressor(max_iter=1000, random_state=2024)],
    #    'regressor__hidden_layer_sizes': [(100, 50), (50, 50), (100, 50, 25, 50, 100)],
    #},
    {
        'regressor': [XGBRegressor(random_state=2024)],
        'regressor__n_estimators': [100, 200, 300],
        'regressor__learning_rate': [0.01, 0.05, 0.1],
        'regressor__max_depth': [3, 6, 10],
        'regressor__subsample': [0.8, 1.0],
        'regressor__colsample_bytree': [0.8, 1.0]
    }
]

# GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(pipeline, param_grid, cv=7, scoring='neg_mean_absolute_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=7,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('regio1',
                                                                         OneHotEncoder(),
                                                                         ['regio1']),
                                                                        ('bool_cols',
                                                                         'passthrough',
                                                                         ['balcony',
                                                                          'garden']),
                                                                        ('num_cols',
                                                                         'passthrough',
                                                                         ['yearConstructed',
                                                                          'noRooms',
                                                                          'livingSpace'])])),
                                       ('regressor', LinearRegression())]),
             n_jobs=-1,
             param_grid=[{'regressor': [XGBRegressor(base_score=No...
                                                     missing=nan,
                                                     monotone_constraints=None,
                                                     multi_strategy=None,
                                                     n_estimators=None,
                                                     n_jobs=None,
                                                     num_parallel_tree=None,
                                                     random_state=2024, ...)],
                          'regressor__colsample_bytree': [0.8, 1.0],
                          'regressor__learning_rate': [0.01, 0.05, 0.1],
                          'regressor__max_depth': [3, 6, 10],
                          'regressor__n_estimators': [100, 200, 300],
                          'regressor__subsample': [0.8, 1.0]}],
             scoring='neg_mean_absolute_error')

In [74]:
# Best model and parameters
print("Best Parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_

# Predictions
y_pred = best_model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")

# Visualizing the predictions vs actual values
#plt.figure(figsize=(10, 6))
#sns.scatterplot(x=y_test, y=y_pred)
#plt.title("Predictions vs Actual Values")
#plt.xlabel("Actual Base Rent")
#plt.ylabel("Predicted Base Rent")
#plt.show()
#

Best Parameters: {'regressor': XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=2024, ...), 'regressor__colsample_bytree': 0.8, 'regressor__learning_rate': 0.05, 'regressor__max_depth': 6, 'regressor__n_estimators': 200, 'regressor__subsample': 1.0}
Mean Absolute Error: 149.21706540425618
Mean Squared Error: 637

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Assuming immo_df is your DataFrame

# Step 1: Preprocess the data
# Convert 'balcony' and 'garden' to boolean
immo_df['balcony'] = immo_df['balcony'].astype(bool)
immo_df['garden'] = immo_df['garden'].astype(bool)

# Define the target variable (baseRent)
y = immo_df['baseRent']

# Features to be used in the model (excluding baseRent)
X = immo_df[['regio1', 'balcony', 'baseRent', 'yearConstructed', 'noRooms', 'livingSpace', 'garden']]

# Step 2: One-hot encoding for 'regio1' and numerical transformation
# Define preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('regio1', OneHotEncoder(), ['regio1']),  # One-hot encode 'regio1'
        ('num', 'passthrough', ['yearConstructed', 'noRooms', 'livingSpace'])  # Keep numerical features as is
    ])

# Step 3: Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Define the model and GridSearchCV
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# Define the parameter grid for RandomForestRegressor
param_grid = {
    'regressor__n_estimators': [50, 100, 200],
    'regressor__max_depth': [None, 10, 20, 30],
    'regressor__min_samples_split': [2, 5, 10]
}

# Perform GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Step 5: Evaluate the model
best_model = grid_search.best_estimator_

# Predict on the test set
y_pred = best_model.predict(X_test)

# Evaluate using MAE and MSE
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")

# Step 6: Visualize the prediction results
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=y_pred)
plt.xlabel('True baseRent')
plt.ylabel('Predicted baseRent')
plt.title('True vs Predicted baseRent')
plt.show()

# Optional: Residual plot
residuals = y_test - y_pred
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_pred, y=residuals)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Predicted baseRent')
plt.ylabel('Residuals')
plt.title('Residuals vs Predicted baseRent')
plt.show()


In [ ]:
import joblib

# Save the best model to disk
best_model_filename = 'best_random_forest_model.joblib'
joblib.dump(grid_search.best_estimator_, best_model_filename)

print(f"Best model saved as {best_model_filename}")

In [ ]:
# Load the saved model
loaded_model = joblib.load(best_model_filename)

# Use the loaded model for predictions
y_pred_loaded = loaded_model.predict(X_test)